In [ ]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef, make_scorer
from statistics import stdev, variance, mean
import torch
from torch import nn, optim
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
random.seed(0)

In [ ]:
species="Mus_musculus"

In [ ]:
species_dict={"Mus_musculus":"mouse", "Saccharomyces_cerevisiae":"yeast", "Arabidopsis_thaliana":"arabi"}

In [ ]:
client_set=set(SeqIO.index("../fig1/result/drllps_client_clstr_Homo_sapiens.fasta", "fasta").keys())
nonllps_set=set(SeqIO.index("../fig1/result/drllps_nonllps_clstr_Homo_sapiens.fasta", "fasta").keys())
    
mat=np.load("../fig2/embedding/PTT5XLU50_human.npy", allow_pickle=True)
mat=mat.item()

list_client=[]
list_nonllps=[]
client_id=[]
nonllps_id=[]
for k in mat.keys():
    if k in nonllps_set:
        list_nonllps.append(mat[k])
        nonllps_id.append(k)
    elif k in client_set:
        list_client.append(mat[k])
        client_id.append(k)

In [ ]:
x = np.array(list_client + list_nonllps)
y = np.array([True]*len(list_client) + [False]*len(list_nonllps))
ids = np.array(client_id + nonllps_id)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, path='checkpoint_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:  
            self.best_score = score   
            self.checkpoint(val_loss, model)  
        elif score <= self.best_score:  
            self.counter += 1   
            if self.verbose:  
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')   
            if self.counter >= self.patience:  
                self.early_stop = True
        else:  
            self.best_score = score  
            self.checkpoint(val_loss, model)  
            self.counter = 0  
            
    def checkpoint(self, val_loss, model):
        if self.verbose:  
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)  
        self.val_loss_min = val_loss  
        
def training_loop(n_epochs, optimizer, model, loss, mask_train, x_train,  y_train):
    loss=loss
    
    n_samples=x_train.shape[0]
    n_val=int(n_samples*0.2)

    shuffled_ind=torch.randperm(n_samples)

    train_ind=shuffled_ind[:-n_val] 
    val_ind=shuffled_ind[-n_val:]
    
    x_val=x_train[val_ind]
    y_val=y_train[val_ind]
    
    x_train=x_train[train_ind]
    y_train=y_train[train_ind]
    
    x_train=x_train
    y_train=y_train
    
    x_val=x_val
    y_val=y_val

    patience=10
    earlystopping = EarlyStopping(patience=patience, verbose=False)
    for epoch in range(1, n_epochs+1):
        model.train()
        
        y_train_pred=model.forward(x_train)
        loss_train=loss(y_train_pred, y_train)
        
        model.eval()
        with torch.no_grad():
            y_val_pred=model.forward(x_val)
            loss_val=loss(y_val_pred, y_val)

        earlystopping(loss_val, model) 
        if earlystopping.early_stop: 
            break
            
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
class FNN2(nn.Module):
    def __init__(self, embeddings_dim=1024, dropout=0.25):
        super(FNN2, self).__init__()

        self.linear = nn.Sequential(
            nn.Linear(embeddings_dim, 32),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(32,2)
        )


    def forward(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        o = self.linear(x)  
        return o
    

class NN(BaseEstimator, ClassifierMixin):
    def __init__(self, n_epochs=500, lr=0.03):
        self.n_epochs = n_epochs
        self.lr = lr
        self.model = None
        self.optim = None
        self.loss = nn.CrossEntropyLoss()

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        X_tensor = torch.tensor(X, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.long)
        n_dim = X_tensor.shape[1]
        self.model=FNN2(embeddings_dim=n_dim)
        self.optim = optim.Adam(self.model.parameters(), lr=self.lr)
        training_loop(
            n_epochs=self.n_epochs,
            optimizer=self.optim,
            model=self.model,
            loss=self.loss,
            mask_train=None,
            x_train=X_tensor,
            y_train=y_tensor,
        )
        return self

    def predict(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float)
            self.model.eval()
            y_pred = self.model(X_tensor)
            _, predicted = torch.max(y_pred, 1)
            return predicted.numpy()

    def predict_proba(self, X):
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float)
            self.model.eval()
            y_pred = self.model(X_tensor)
            probas = nn.Softmax(dim=1)(y_pred)
            return probas.numpy()

In [ ]:
estimators = [
    ('nn', NN(lr=0.01)),
    ('rf', RandomForestClassifier(max_depth=20, max_features="sqrt", class_weight="balanced",n_estimators=200, n_jobs=50)),
    ('svm', make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True, gamma="auto"))),
    ('hgboost', HistGradientBoostingClassifier(learning_rate=0.1, max_leaf_nodes=63, min_samples_leaf=80, class_weight="balanced"))
]
cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
model=StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(class_weight="balanced"), n_jobs=-1, cv=cv
)

In [ ]:
model.fit(x,y)

In [ ]:
def eval_species(species, model):
    client_set=set(SeqIO.index("result/drllps_client_clstr_"+species+".fasta", "fasta").keys())
    nonllps_set=set(SeqIO.index("result/drllps_nonllps_clstr_"+species+".fasta", "fasta").keys())
    no_homolog_set=set(SeqIO.index("result/"+species_dict[species]+"_no_human_homolog.faa", "fasta").keys())

    mat=np.load("data/PTT5XLU50_"+species_dict[species]+".npy", allow_pickle=True)
    mat=mat.item()

    list_client_species=[]
    list_nonllps_species=[]
    for k in mat.keys():
        if k in nonllps_set:
            if k in no_homolog_set:
                list_nonllps_species.append(mat[k])
        elif k in client_set:
            if k in no_homolog_set:
                list_client_species.append(mat[k])
                
    print("# client:{}, # non-llps:{}".format(len(list_client_species),  len(list_nonllps_species)))
    
    x_species = np.array(list_client_species + list_nonllps_species)
    y_species = np.array([True]*len(list_client_species) + [False]*len(list_nonllps_species))
    y_pred=model.predict_proba(x_species)[:,1]
    y_pred_bi=model.predict(x_species)
    fpr, tpr, thresholds = roc_curve(y_species, y_pred)
    roc_auc=roc_auc_score(y_species, y_pred)
    precision, recall, thresholds = precision_recall_curve(y_species, y_pred)
    pr_auc = auc(recall, precision)
    mcc=matthews_corrcoef(y_species, y_pred_bi)
    print("ROCAUC={:.3f}, PRAUC={:.3f}, MCC={:.3f}".format(roc_auc, pr_auc, mcc))
    
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    
    ax.plot(
        fpr,
        tpr,
        color="b",
        label=r"Seq2Phase (ROC-AUC = %0.3f)" % (roc_auc),
        lw=2,
        alpha=0.8,
    )
    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", alpha=0.3)
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title=species,
    )
    
    sp_name=species.split("_")[0][0]+"."+species.split("_")[1]
    ax.set_title(sp_name, fontsize=15, fontstyle='italic')
    ax.legend(loc="lower right")
    ax.set_xlabel("False positive rate", fontsize=13)
    ax.set_ylabel("True positive rate", fontsize=13)
    plt.savefig("result/fig5_"+species_dict[species]+"_roc.pdf")
    plt.show()
    
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    
    ax.plot(
        recall,
        precision,
        color="b",
        label=r"Seq2Phase (PR-AUC = %0.3f)" % (pr_auc),
        lw=2,
        alpha=0.8,
    )
    
    ax.set_title(sp_name, fontsize=15, fontstyle='italic')
    ax.legend(loc="lower right")
    ax.set_xlabel("Recall", fontsize=13)
    ax.set_ylabel("Precision", fontsize=13)
    plt.savefig("result/"+species_dict[species]+"_pr.pdf")
    plt.show()
    
    
    list_non_client_species=[]
    for k in mat.keys():
        if k in client_set:
            continue
        else:
            list_non_client_species.append(mat[k])
                
    print("# non-client:{}".format(len(list_non_client_species)))
    
    x_species = np.array(list_non_client_species)
    y_pred_bi=model.predict(x_species)
    
    print("# predicted clients = {}".format(len(y_pred_bi[y_pred_bi == True])))

In [ ]:
eval_species(species, model)

In [ ]:
eval_species("Saccharomyces_cerevisiae", model)

In [ ]:
eval_species("Arabidopsis_thaliana", model)